<h1>Grafo delle co-authorship tra gli articoli del corso di matematica</h1>
<p>Dal dataset degli articoli di matematica vengono filtrati solamente quelli dei professori che attualmente sono nella lista del dipartimento di informatica.</p>

In [12]:
from pyvis.network import Network
import networkx as nx
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import re
from collections import Counter

#from src.grafico_informatica_prima_versione import df_professori_articoli

<h2>Caricamento dei dati</h2>

<h3>Dataset degli articoli di fisica</h3>
<p>In formato csv e convertito a dataframe</p>

In [13]:
# Carico il dataset
df_articoli_matematica = pd.read_csv("../data/matematica.csv")

<h3>Creazione della lista dei professori di informatica</h3>
<p>I nomi sono stati copiati dal sito dell'università e incollati in un file di testo.</p>

In [14]:
# Carico la lista dei professori dal file txt
with open("../data/professori_informatica.txt", "r", encoding="utf-8") as f:
    professori_informatica = [riga.strip() for riga in f]

with open("../data/professori_matematica.txt", "r", encoding="utf-8") as f:
    professori_matematica = [riga.strip() for riga in f]

with open("../data/professori_scienze_tecnologie_multimediali.txt", "r", encoding="utf-8") as f:
    professori_tecnologie_multimediali = [riga.strip() for riga in f]

def normalize_prof(name):
    cognome, nome = [x.strip() for x in name.split(',')]
    return f"{cognome} {nome}"

def normalize_list(lista_professori):
    lista_professori = set(normalize_prof(n) for n in lista_professori)
    return list(lista_professori)


professori_informatica = normalize_list(professori_informatica)
professori_matematica = normalize_list(professori_matematica)
professori_tecnologie_multimediali = normalize_list(professori_tecnologie_multimediali)

#print(professori_informatica)

#print("Prof di matematica: \n", professori_matematica)

print("Prof di tecnologie multimediali: \n", professori_tecnologie_multimediali)

Prof di tecnologie multimediali: 
 ['Raggiotto Francesco', 'Marinelli De Marco Giuseppe', 'Piciarelli Claudio', 'Martinel Niki', 'Micheloni Christian', 'Fusiello Andrea', 'Fontana Federico', 'Rotondi Elena', 'Burigat Stefano', 'Toma Andrea', 'Ballis Demis', 'Piccolo Chiara', 'Cossettini Luca', 'Cevolin Guglielmo', 'Farinosi Manuela', 'Mizzaro Stefano', 'Della Mea Vincenzo', 'Foresti Gian Luca', 'Fasino Dario', 'Zucco Rodolfo', 'Atzori Paolo Cesare', 'Rossitti Marco', 'Liessi Davide', 'Snidaro Lauro', 'Chittaro Luca', 'Drioli Carlo', 'Salvati Daniele', 'Di Iorio Marcela Beatriz', 'Comini Marco', 'Calabretto Roberto', 'Roitero Kevin']


<h2>Funzione per estrarre gli autori dalla relativa colonna del dataframe</h2>
<p>Gli autori sono memorizzati come: "Cognome, Nome (ID); ..."<br>
Quindi vogliamo trasformarli in "Cognome Nome"
</p>

In [15]:
def pulisci_autori(author_string):
    if pd.isna(author_string):
        return []
    
    autori = []
    for a in author_string.split(";"):
        a = a.strip()
        # prendo la parte prima di "("
        a = a.split("(")[0].strip()
        
        # separo Cognome, Nome
        if "," in a:
            cognome, nome = a.split(",", 1)
            autori.append(f"{cognome.strip()} {nome.strip()}")
        else:
            autori.append(a)
    return autori

<h2>Creazione del grafo</h2>
<p>Viene anche salvato in formato pickle in modo tale che possa essere utilizzato in altri notebook</p>

In [16]:
G_prof_informatici = nx.Graph()
G_prof_matematica = nx.Graph()
G_prof_tecnologie= nx.Graph()

def creaGrafoProf(G,lista, path_pickle):
    for autori_string in df_articoli_matematica["Author full names"]:
        lista_autori = pulisci_autori(autori_string)

        # Filtra solo i professori del dipartimento
        autori_prof = [a for a in lista_autori if a in lista]

        # Aggiungi edge tra tutte le coppie di professori che hanno co-firmato
        for i in range(len(autori_prof)):
            for j in range(i + 1, len(autori_prof)):
                a1, a2 = autori_prof[i], autori_prof[j]

                if G.has_edge(a1, a2):
                    G[a1][a2]["weight"] += 1
                else:
                    G.add_edge(a1, a2, weight=1)
    with open(path_pickle, "wb") as f:
        pickle.dump(G, f)

creaGrafoProf(G_prof_informatici,professori_informatica ,"./grafi_pickle/articoli_matematica/G_prof_informatici.pkl")
creaGrafoProf(G_prof_matematica, professori_matematica,"./grafi_pickle/articoli_matematica/G_prof_matematica.pkl")
creaGrafoProf(G_prof_tecnologie, professori_tecnologie_multimediali,"./grafi_pickle/articoli_matematica/G_prof_tecnologie.pkl")

<h3>Dati del grafo</h3>
<ul>
    <li>Numero di nodi (professori coinvolti)</li>
    <li>Numero di archi (co-authorship)</li>
    <li>Per ogni professore i relativi collaboratori</li>
</ul>

In [17]:
def stampaDatiGrafico(g):
    print("Numero di nodi (professori coinvolti):", g.number_of_nodes())
    print("Numero di archi (co-autorship):", g.number_of_edges())

    # Mostra i vicini di un professore, se vuoi
    for n in g.nodes():
        print(n, "co-autore con:", list(g.neighbors(n)))
#stampaDatiGrafico(G_prof_informatici)
#stampaDatiGrafico(G_prof_matematica)
stampaDatiGrafico(G_prof_tecnologie)


Numero di nodi (professori coinvolti): 17
Numero di archi (co-autorship): 21
Piciarelli Claudio co-autore con: ['Foresti Gian Luca', 'Martinel Niki', 'Micheloni Christian']
Foresti Gian Luca co-autore con: ['Piciarelli Claudio', 'Toma Andrea', 'Drioli Carlo', 'Fontana Federico', 'Fusiello Andrea', 'Micheloni Christian', 'Martinel Niki', 'Salvati Daniele', 'Snidaro Lauro']
Toma Andrea co-autore con: ['Drioli Carlo', 'Foresti Gian Luca', 'Salvati Daniele']
Drioli Carlo co-autore con: ['Toma Andrea', 'Foresti Gian Luca', 'Salvati Daniele']
Martinel Niki co-autore con: ['Micheloni Christian', 'Foresti Gian Luca', 'Piciarelli Claudio']
Micheloni Christian co-autore con: ['Martinel Niki', 'Foresti Gian Luca', 'Piciarelli Claudio', 'Snidaro Lauro']
Fontana Federico co-autore con: ['Foresti Gian Luca']
Mizzaro Stefano co-autore con: ['Roitero Kevin', 'Della Mea Vincenzo']
Roitero Kevin co-autore con: ['Mizzaro Stefano', 'Della Mea Vincenzo']
Della Mea Vincenzo co-autore con: ['Roitero Kevin', 

<h3>Visualizzazione del grafo tramite libreria Pyvis</h3>
<p>Il file viene salvato in formato .html nella relativa cartella</p>

In [18]:
# Crea un grafo pyvis
def creaGrafo(G,output_path):
    net = Network(height="750px", width="100%", bgcolor="black", font_color="white", notebook=True, cdn_resources="remote", filter_menu=True)

    # Forza fisica per un layout più bello
    net.force_atlas_2based()

    # Aggiungi nodi e archi da NetworkX a PyVis
    for node in G.nodes():
        net.add_node(
            node,
            label=node,
            title=node,
            color="#4A90E2",   # colore nodi
            shape="dot",
            size=15
        )

    for u, v, data in G.edges(data=True):
        weight = data.get("weight", 1)
        net.add_edge(
            u,
            v,
            value=weight,
            title=f"Co-authored papers: {weight}"
        )
    # Salva in HTML
    net.write_html(output_path)
    #print("File generato:", output_path)
    return net


In [19]:

output_path_informatica= "../html/grafi_co-authorship/artcoli_matematica/professori-informatica.html"
creaGrafo(G_prof_informatici, output_path_informatica).show(output_path_informatica)

../html/grafi_co-authorship/artcoli_matematica/professori-informatica.html


In [20]:
output_path_matematica= "../html/grafi_co-authorship/artcoli_matematica/professori-matematica.html"
creaGrafo(G_prof_matematica, output_path_matematica).show(output_path_matematica)

../html/grafi_co-authorship/artcoli_matematica/professori-matematica.html


In [21]:
output_path_tecnologie= "../html/grafi_co-authorship/artcoli_matematica/professori-tecnologie-multimediali.html"
creaGrafo(G_prof_tecnologie, output_path_informatica).show(output_path_tecnologie)

../html/grafi_co-authorship/artcoli_matematica/professori-tecnologie-multimediali.html
